## USPTO Company Information Crawler
##### Last Update: 2022-08-03
#### Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Python Version: Python 3.10.4 (ipykernel)
#### Chrome Version: Chrome 103.0.5060.134 (64-bit)
#### Chrome Driver Version: ChromeDriver 103.0.5060.134

#### Crawled Website
BSE India MAIN: https://uspto.report/TM/

Description: This is USPTO(United States Patent and Trademark Office) company information crawler. USPTO updates all company, individual, business information that requested trademark registration on their system. Appearantly, they upload data on Thursday and Friday. The website is quite valuable as they have information starting from 1973. Approximately, they have more than 2 million request data. To use automative crawling, VPN IP for United States is required for entrance. I used Nord VPN for the crawling and it was fine. For crawling, as long as the website has not been modified, the code below must be run properly if and only if it is run in an order. Moreover, environment setting must be fixed based on user's computer/server setting and location beforehead. Since the purpose of this code is to collect general information of company, business, or individual, this does not collect any detailed financial information of companies.

### 1. Importing useful open source librabries - utilized BS4 and Selenium Web Driver for crawling

In [7]:
import bs4
import time
import csv 
import pandas as pd 
from platform import python_version
import requests
import lxml 
import xlrd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import csv 
import pandas as pd 
from platform import python_version
import xlsxwriter
from bs4 import BeautifulSoup

### 2. Environment
#### - Chromdriver must be located in local/cloud PC folder.
#### - One can download proper version at: chromedriver.chromium.org/downloads
#### - Path should be modified based on local environment

In [8]:
path = r'C:\Users\user\Desktop\자료\chromedriver'

In [9]:
print(python_version())
print(pd.__version__)

3.9.12
1.4.2


#### - Initial Starting point must be updated

In [19]:
initial = 97510376

## 3. Main Crawler
#### - Uncollectable or unavailable data was collected as "" for convenience.
#### - Lots of try-except function is used to avoid error caused by non-existing pages or information.
#### - The order of append must  be depend on the order of column name assigned above.
#### - Data Cleansing might be required after crawling sometiems, but it will not be tough as one knows how to use excel.

### File Setting
#### - xslx file name must be checked based on user preference.
#### - xlsx file will be newly added on the folder where ipynb file is located.
#### - Column can be changed if it is required, but the main crawler code must be also modified for accuracy.
#### - The way we insert the information on xlsx file is different from csv file.
#### - Closing workbook is necessary - it will not save all the data on the sheet if it is not closed at the end.
#### - DO NOT RUN THIS CODE AGAIN AFTER CRAWLING OR THE BLANK FILE WILL REPLACE CURRENT FILE.
#### - Maximum number of range should be less than 200,000

In [ ]:
workbook = xlsxwriter.Workbook('USPTO_Crawling_Main.xlsx')
worksheet = workbook.add_worksheet()
a = 0
row = 0
column = 0

content = ['Serial Number','Legal Entity State','Legal Entity Type','Company Name','Mark Literal','Mark Type','Purpose','Class Status','Primary US Classes - 1','Primary US Classes - 2','Primary US Classes - 3','Primary US Classes - 4','Primary US Classes - 5','Primary US Classes - 6','Primary International Class','Primary International Class Description','Address','Source','Date']
for item in content :
    worksheet.write(row, column, item)
    column += 1
row += 1

base_url = "https://uspto.report/TM/"
driver = webdriver.Chrome(path)
driver.get(base_url)
time.sleep(3)
failList = []

for i in reversed(range(97200000,initial)):
    try:
        try:
            driver.get(base_url + str(i))
        except:
            time.sleep(3)
            driver.get("https://www.google.com/")
            time.sleep(3)
        serial = i
        a = i
        column = 0
        info = []
        time.sleep(2.5)
        try:
            head = driver.find_element(By.XPATH, value='/html/body/div[2]/div/h1').text
        except:
            head = "Trademark Filing"
        if head == "Trademark Filing":
            print("NA")
        else:
            #serial number
            info.append(serial)

            #state
            try:
                state = driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[4]/td[2]').text
            except:
                try:
                    state = driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table/tbody/tr[3]/td[2]').text
                except:
                    try:
                        state = driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[2]/td[2]').text
                    except:
                        try:
                            state = driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[4]/td[2]').text
                        except:
                            try:
                                state = driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[5]/div[2]/table/tbody/tr[4]/td[2]').text
                            except:
                                try:
                                    state = driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table/tbody/tr[4]/td[2]').text
                                except:
                                    state = "NA"

            #legal entity state
            info.append(state)

            #legal entity type
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[3]/td[2]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table[1]/tbody/tr[3]/td[2]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[3]/td[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[5]/div[2]/table/tbody/tr[2]/td[2]').text)
                        except:
                            info.append("Business")

            #Company Name
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[1]/td[2]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/nav/ol/li[2]/a/span').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/h2').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/nav/ol/li[2]/a/span').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[1]/td[2]/a').text)
                            except:
                                try:
                                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table[2]/tbody/tr[1]/td[2]').text)
                                except:
                                    try:
                                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table[1]/tbody/tr[1]/td[2]').text)
                                    except:
                                        info.append("Failed")

            #Mark Literal
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/h1').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/nav/ol/li[3]/a/span').text)
                except:
                    info.append("Failed")

            #Mark Type
            info.append("Trademark")

            #Purpose
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/p[2]/span').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/p').text)
                except:
                    info.append("Trademark")

            #status
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/div[2]/div[3]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/div[1]/div[3]').text)
                except:
                    info.append("Status not available")

            #industry
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[1]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[1]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[11]/td[2]/ul/li[1]').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
                            except:
                                try:
                                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                                except:
                                    info.append("Business")

            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[1]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[11]/td[2]/ul/li[2]').text)
                            except:
                                try:
                                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                                except:
                                    info.append("Business")

            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[3]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[2]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[11]/td[2]/ul/li[3]').text)
                            except:
                                try:
                                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                                except:
                                    info.append("Business")

            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[4]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[2]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[11]/td[2]/ul/li[4]').text)
                            except:
                                try:
                                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                                except:
                                    info.append("Business")

            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[5]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[3]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[11]/td[2]/ul/li[5]').text)
                            except:
                                try:
                                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                                except:
                                    try:
                                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[1]').text)
                                    except:
                                        info.append("Business")

            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[6]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[3]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[11]/td[2]/ul/li[5]').text)
                            except:
                                try:
                                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                                except:
                                    try:
                                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[1]').text)
                                    except:
                                        info.append("Business")
            #international
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[1]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[11]/td[2]/ul/li[5]').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                            except:
                                info.append("Business")

            #international description
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[10]/td[2]/ul/li[2]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table[1]/tbody/tr[9]/td[2]/ul/li[2]').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[10]/div[2]/table[2]/tbody/tr[2]/td[2]').text)
                            except:
                                info.append("Business")


            #address
            try:
                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[2]/td[2]').text)
            except:
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[5]/div[2]/table[1]/tbody/tr[2]/td[2]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[2]/td[2]').text)
                    except:
                        try:
                            info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[6]/div[2]/table/tbody/tr[3]/td[2]').text)
                        except:
                            try:
                                info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[5]/div[2]/table/tbody/tr[2]/td[2]').text)
                            except:
                                try:
                                    info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[5]/div[2]/table/tbody/tr[3]/td[2]').text)
                                except:
                                    try:
                                        info.append(driver.find_element(By.XPATH, value='/html/body/div[2]/div/div[4]/div[2]/table/tbody/tr[4]/td[2]').text)
                                    except:
                                        info.append(state)
            info.append("USPTO")
            info.append("2022-07-26")
            print(info)
            for item in info :
                worksheet.write(row, column, item)
                column += 1
            row += 1
            print("End Point:")
            print(a)
    except:
        time.sleep(3)
        driver.get("https://www.google.com/")
        time.sleep(3)
        print("fail")
workbook.close()
print("Finished")

C:\Users\user\AppData\Local\Temp\ipykernel_14388\3888361251.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


[97510375, 'SIOPUE', 'Individual', 'Kong, Debao', 'SIOPUE', 'Trademark', "Mark For: SIOPUE™ trademark registration is intended to cover the categories of earrings; Body jewelry; Bracelets; Charms for jewelry; Children's jewelry; Hair jewelry in the nature of jewelry for use in the hair; Key chains; Key rings; Necklaces; Women's jewelry.", 'LIVE APPLICATION Awaiting Examination', '002: Receptacles', '027: Horological Instruments', '028: Jewelry and Precious-metal Ware', '050: Merchandise Not Otherwise Classified', "Earrings; Body jewelry; Bracelets; Charms for jewelry; Children's jewelry; Hair jewelry in the nature of jewelry for use in the hair; Key chains; Key rings; Necklaces; Women's jewelry", "Earrings; Body jewelry; Bracelets; Charms for jewelry; Children's jewelry; Hair jewelry in the nature of jewelry for use in the hair; Key chains; Key rings; Necklaces; Women's jewelry", '014 - Primary Class', '(Jewelry) Precious metals and their alloys and goods in precious metals or coated t

NA
[97510365, 'YCIUSE', 'Individual', 'Ouyang, Ji', 'YCIUSE', 'Trademark', 'Mark For: YCIUSE™ trademark registration is intended to cover the categories of bento boxes; Dining plates of silica gel; Dinnerware, namely, plates, cups and saucers; Dishers; Holders for toilet paper; Household containers for foods; Meal trays; Plastic bath racks; Towel rails; Towel rails and rings.', 'LIVE APPLICATION Awaiting Examination', '002: Receptacles', '013: Hardware, Plumbing and Steamfitting Supplies', '023: Cutlery, Machinery, Tools and Parts Thereof', '029: Brooms, Brushes and Dusters', '030: Crockery, Earthenware and Porcelain', '033: Glassware', '021 - Primary Class', '(Housewares and glass) Household or kitchen utensils and containers (not of precious metal or coated therewith); combs and sponges; brushes (except paint brushes); brush-making materials; articles for cleaning purposes; steel wool; un-worked or semi-worked glass (except glass used in building); glassware, porcelain and earthenwar

NA
[97510355, 'TAIWAN', 'Corporation', 'LUKAS BIOMEDICAL INC.', 'FRESHO2', 'Trademark', 'blusher; Cosmetics; Lipstick; Mascara; Concealers for facial lines and wrinkles; Eyebrow pencils; Eyeshadow palettes; Lip balm; Liquid foundation (mizu-oshiroi); Pressed face powder.', 'LIVE APPLICATION Awaiting Examination', '001: Raw or Partly Prepared Materials', '001: Raw or Partly Prepared Materials', '004: Abrasives and Polishing Materials', '004: Abrasives and Polishing Materials', '006: Chemicals and Chemical Compositions', '006: Chemicals and Chemical Compositions', '001: Raw or Partly Prepared Materials', '004: Abrasives and Polishing Materials', "3F., No. 2, Ln. 265, Sec. 4, Xinyi Rd. Da'an Dist. Taipei City TAIWAN 106066", 'USPTO', '2022-07-26']
End Point:
97510355
[97510354, 'DELAWARE', 'Limited Liability Company', 'Mean Kids, LLC', 'KRYPT KITTIES', 'Trademark', 'Mark For: KRYPT KITTIES™ trademark registration is intended to cover the categories of downloadable audio and video recordin

[97510343, 'CALIFORNIA', 'Limited Liability Company', 'Krymis Productions LLC', 'STYLIZED WORDS WITH CROWN DESIGN ELEMENT', 'Trademark', 'restaurant; Restaurant and bar services, including restaurant carryout services; Restaurant services; Restaurant services, including sit-down service of food and take-out restaurant services; Restaurant services, namely, providing of food and beverages for consumption on and off the premises; Bar and restaurant services; Take-out restaurant services.', 'LIVE APPLICATION Awaiting Examination', '100: Miscellaneous', '100: Miscellaneous', '101: Advertising and Business', '101: Advertising and Business', '100: Miscellaneous', '100: Miscellaneous', '100: Miscellaneous', '101: Advertising and Business', '1299 Meadowbrook Ave Los Angeles, CALIFORNIA UNITED STATES 90019', 'USPTO', '2022-07-26']
End Point:
97510343
NA
[97510341, 'FLORIDA', 'Limited Liability Company', '4R VENTURES, LLC', '4ROOTS FARM', 'Trademark', 'promoting the interests of sustainable farm

...

[97507487, 'DELAWARE', 'Corporation', 'SuperPlastic, Inc.', "LIL' ILL", 'Trademark', "Mark For: LIL' ILL™ trademark registration is intended to cover the categories of visual recordings and audiovisual recordings featuring music and animation; Downloadable and recorded computer programs for video and computer games; Interactive video game programs; Audio and visual recordings featuring live-action entertainment, animated characters; Downloadable virtual goods, namely, multimedia files containing photos, artwork, drawings, audio, video, images, text, animations, still images, moving images, music, video relating to animated characters authenticated by non-fungible tokens (NFTs); downloadable digital media, namely, digital collectibles, digital tokens and crypto-collectibles being photos, artwork, drawings, audio, video, images, text, animations, still images, moving images, music, video games, playing cards, card games, tickets, digital multimedia content, intellectual property, written

[97507485, 'DELAWARE', 'Limited Liability Company', 'Standard Fiber, LLC', 'PETNOLOGY', 'Trademark', 'Mark For: PETNOLOGY™ trademark registration is intended to cover the category of beds for household pets.', 'LIVE APPLICATION Awaiting Examination', '002: Receptacles', '013: Hardware, Plumbing and Steamfitting Supplies', '022: Games, Toys and Sporting Good', '025: Locks and Safes', '032: Furniture and Upholstery', '050: Merchandise Not Otherwise Classified', '020 - Primary Class', '(Furniture and articles not otherwise classified) Furniture, mirrors, picture frames; goods (not included in other classes) of wood, cork, reed, cane, wicker, horn, bone, ivory, whalebone, shell, amber, mother-of-pearl, meerschaum and substitutes for all these materials, or of plastics.', 'Suite 200 577 Airport Boulevard Burlingame, CALIFORNIA UNITED STATES 94010', 'USPTO', '2022-07-26']
End Point:
97507485
[97507484, 'DELAWARE', 'Corporation', 'Kernal Biologics', 'KERNAL', 'Trademark', 'Mark For: KERNAL™ t

[97507477, 'FLORIDA', 'Limited Liability Company', 'Zavidah, LLC', 'ZAVIDAH WELLNESS', 'Trademark', 'Mark For: ZAVIDAH WELLNESS™ trademark registration is intended to cover the category of nutritional supplements in the form of pure aloe vera extract.', 'LIVE APPLICATION Awaiting Examination', '005: Adhesives', '006: Chemicals and Chemical Compositions', '018: Medicines and Pharmaceutical Preparations', '044: Dental, Medical and Surgical Appliances', '046: Foods and Ingredients of Foods', '051: Cosmetics and Toilet Preparations', '005 - Primary Class', '(Pharmaceuticals) Pharmaceutical, veterinary and sanitary preparations; dietetic substances adapted for medical use, food for babies; plasters, materials for dressings; material for stopping teeth, dental wax; disinfectants; preparations for destroying vermin; fungicides, herbicides.', '848 Brickell Key Dr. Apt. 2906 Miami, FLORIDA UNITED STATES 33131', 'USPTO', '2022-07-26']
End Point:
97507477
[97507476, 'CHINA', 'Limited Company (ltd

[97507469, 'TEXAS', 'Limited Liability Company', 'Williams charlotte A', 'CHARLOTTE ANN WILLIAMS', 'Trademark', 'Mark For: CHARLOTTE ANN WILLIAMS™ trademark registration is intended to cover the category of marketing consulting services relating to long-term business strategy development for non-profit organizations.', 'LIVE APPLICATION Awaiting Examination', '100: Miscellaneous', '100: Miscellaneous', '101: Advertising and Business', '101: Advertising and Business', '102: Insurance and Financial', '102: Insurance and Financial', '100: Miscellaneous', '101: Advertising and Business', '5680 HIGHWAY 6 STE 119 MISSOURI CITY, TEXAS UNITED STATES 77459', 'USPTO', '2022-07-26']
End Point:
97507469
[97507468, 'UNITED KINGDOM', 'Corporation', 'Laflex Equestrian International Limited', 'FLEXARS', 'Trademark', 'leggings.', 'LIVE APPLICATION Awaiting Examination', '022: Games, Toys and Sporting Good', '022: Games, Toys and Sporting Good', '039: Clothing', '039: Clothing', '022: Games, Toys and Sp

[97507459, 'CHINA', 'Limited Company (ltd.)', 'Yiwu Oujing Electronic Commerce Co., Ltd', 'YWZDS', 'Trademark', 'Mark For: YWZDS™ trademark registration is intended to cover the categories of bangles; Ankle bracelets; Bead bracelets; Bib necklaces; Bracelets made of embroidered textile; Charity bracelets; Charm bracelets; Choker necklaces; Cuff bracelets; Identification bracelets; Key chains; Neck chains; Stainless steel jewelry bracelets.', 'LIVE APPLICATION Awaiting Examination', '002: Receptacles', '027: Horological Instruments', '028: Jewelry and Precious-metal Ware', '050: Merchandise Not Otherwise Classified', '027: Horological Instruments', '027: Horological Instruments', '014 - Primary Class', '(Jewelry) Precious metals and their alloys and goods in precious metals or coated therewith, not included in other classes; jewelry, precious stones; horological and chronometric instruments.', 'Room 1306, Unit 1, Building 2 Oujing Mingcheng, Choujiang Street Yiwu City, Zhejiang CHINA 32